# FleetSafe EDA

This notebook explores traffic violations and optional weather data to inform targeted training and awareness campaigns.

- Load tickets (2023 required, optional 2022)
- Sanity checks, missing values, distributions
- Temporal patterns
- Violation types, age, gender
- Weather merge (day-level fallback)



In [24]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
from dateutil import parser
import plotly.express as px

# Robust path resolution: search upwards from current working directory
CWD = Path.cwd()

def find_upwards(filename: str, start: Path = CWD, max_levels: int = 5) -> Path | None:
    p = start
    for _ in range(max_levels):
        candidate = p / filename
        if candidate.exists():
            return candidate
        p = p.parent
    return None

p_2023 = find_upwards("Traffic_Tickets_Issued_Window_2023.csv")
p_2022 = find_upwards("Traffic_Tickets_Issued_Window_2022.csv")
p_wx = find_upwards("cleaned_weather_dataset.csv")

if p_2023 is None:
    raise FileNotFoundError("Traffic_Tickets_Issued_Window_2023.csv not found. Place it in the project root.")

# Load
df23 = pd.read_csv(p_2023)
df22 = pd.read_csv(p_2022) if p_2022 is not None else None
wx = pd.read_csv(p_wx) if p_wx is not None else None

print("2023:", p_2023, df23.shape)
if df22 is not None:
    print("2022:", p_2022, df22.shape)
if wx is not None:
    print("Weather:", p_wx, wx.shape)



2023: c:\Users\peter\OneDrive\Documentos\Personal\TrafficTickets\Traffic_Tickets_Issued_Window_2023.csv (2795593, 11)
2022: c:\Users\peter\OneDrive\Documentos\Personal\TrafficTickets\Traffic_Tickets_Issued_Window_2022.csv (2543003, 11)
Weather: c:\Users\peter\OneDrive\Documentos\Personal\TrafficTickets\cleaned_weather_dataset.csv (365, 12)


In [25]:
# Column guesses (edit as needed)
col_dt = next((c for c in df23.columns if c.lower() in ["date", "datetime", "timestamp"]), None)
col_violation = next((c for c in df23.columns if "violation" in c.lower() or "offense" in c.lower()), None)
col_age = next((c for c in df23.columns if "age" in c.lower()), None)
col_gender = next((c for c in df23.columns if "gender" in c.lower() or c.lower() in ["sex"]), None)

print({"datetime": col_dt, "violation": col_violation, "age": col_age, "gender": col_gender})

# Parse datetime
if col_dt:
    df23["datetime"] = pd.to_datetime(df23[col_dt], errors="coerce")
    df23["month"] = df23["datetime"].dt.to_period("M").dt.to_timestamp()
    df23["weekday"] = df23["datetime"].dt.day_name()



{'datetime': None, 'violation': 'Violation Charged Code', 'age': 'Age at Violation', 'gender': 'Gender'}


In [26]:
# Distributions and missing values
print(df23.isna().mean().sort_values(ascending=False).head(20))
if col_violation:
    print(df23[col_violation].value_counts().head(10))
if col_age:
    print(pd.to_numeric(df23[col_age], errors="coerce").describe())
if col_gender:
    print(df23[col_gender].value_counts())



Age at Violation          0.015885
Violation Description     0.000000
Violation Charged Code    0.000000
Violation Year            0.000000
Violation Month           0.000000
Violation Day of Week     0.000000
Gender                    0.000000
State of License          0.000000
Police Agency             0.000000
Court                     0.000000
Source                    0.000000
dtype: float64
Violation Charged Code
1180D      260901
1110A      179236
306B       173658
5091       160804
1180B      139315
4011A      107482
1172A       76176
37512A2     74982
1180D12     73187
1225D       72226
Name: count, dtype: int64
count    2.751185e+06
mean     3.647965e+01
std      1.372628e+01
min      1.600000e+01
25%      2.500000e+01
50%      3.400000e+01
75%      4.500000e+01
max      9.500000e+01
Name: Age at Violation, dtype: float64
Gender
M    2057964
F     695534
C      41828
X        267
Name: count, dtype: int64


In [27]:
# Time series and weekday patterns
if col_dt:
    ts = df23.groupby("month").size().reset_index(name="count")
    fig = px.line(ts, x="month", y="count", title="Monthly Violations (2023)")
    fig.show()

    wd = df23.groupby("weekday").size().reset_index(name="count")
    order = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    wd["weekday"] = pd.Categorical(wd["weekday"], categories=order, ordered=True)
    wd = wd.sort_values("weekday")
    fig2 = px.bar(wd, x="weekday", y="count", title="Violations by Weekday")
    fig2.add_hline(y=wd["count"].mean(), line_dash="dash", line_color="orange")
    fig2.show()



In [29]:
# Violation vs age group heatmap and stacked by gender
if col_age:
    ages = pd.to_numeric(df23[col_age], errors="coerce")
    bins = [0,17,25,35,45,55,65,200]
    labels = ["<=17","18-25","26-35","36-45","46-55","56-65","66+"]
    df23["age_group"] = pd.cut(ages, bins=bins, labels=labels, right=True, include_lowest=True)

if col_violation and col_age:
    hm = df23.groupby(["age_group", col_violation]).size().reset_index(name="count")
    pivot = hm.pivot(index="age_group", columns=col_violation, values="count").fillna(0)
    fig3 = px.imshow(pivot, aspect="auto", title="Heatmap: Age Group vs Violation Type")
    fig3.show()

if col_age and col_gender:
    ag = df23.groupby(["age_group", col_gender]).size().reset_index(name="count")
    fig4 = px.bar(ag, x="age_group", y="count", color=col_gender, barmode="stack", title="Stacked: Age Group by Gender")
    fig4.show()



C:\Users\peter\AppData\Local\Temp\ipykernel_21632\2021778130.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
# Weather join (day-level fallback)
if wx is not None:
    wx_cols = {c.lower(): c for c in wx.columns}
    wx_dt = wx_cols.get("datetime") or wx_cols.get("date")
    wx_cond = wx_cols.get("condition") or wx_cols.get("weather")
    if wx_dt and wx_cond and col_dt:
        wx2 = wx[[wx_dt, wx_cond]].copy()
        wx2[wx_dt] = pd.to_datetime(wx2[wx_dt], errors="coerce")
        wx2["wx_day"] = wx2[wx_dt].dt.date
        df23["day"] = df23["datetime"].dt.date
        merged = df23.merge(wx2[["wx_day", wx_cond]].dropna().drop_duplicates("wx_day"), left_on="day", right_on="wx_day", how="left")
        wc = merged[wx_cond].fillna("Unknown")
        s = merged.groupby(wc).size().reset_index(name="count").sort_values("count", ascending=False)
        fig5 = px.bar(s, x=wx_cond, y="count", title="Weather vs Violations")
        fig5.show()

